In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = '/content/drive/MyDrive/Hotel System/'
df = pd.read_csv(path + 'hotel_train.csv')
df_test = pd.read_csv(path + 'hotel_test.csv')

print(f'Train shape: {df.shape}')
print(f'Test shape: {df_test.shape}')

Train shape: (65229, 25)
Test shape: (32412, 25)


In [3]:
print(df.head())

   id  is_canceled  ...  total_of_special_requests  total_nights
0   0            0  ...                          0             1
1   1            0  ...                          1             2
2   2            0  ...                          0             2
3   3            0  ...                          1             2
4   4            1  ...                          1             3

[5 rows x 25 columns]
